# Apache Spark Tutorial

University of Liège                                                      
Joeri Hermans (joeri.hermans@doct.ulg.ac.be)                         

This notebook will guide the students of "Large-Scale Distributed Systems" through their initial steps with [Apache Spark](https://spark.apache.org). If you would like additional information about the concepts presented in this notebook, please refer to the [Spark Programming Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html).

## Downloading, unpacking, installing and system configuration

In [ ]:
# Download the latest version of Apache Spark.
!rm -r spark-2.2.0-bin-hadoop2.7.tgz
!wget -q http://apache.tt.co.kr/spark/spark-2.2.0/spark-2.2.0-bin-hadoop2.7.tgz

In [ ]:
# Unpack the framework.
!rm -r apache-spark
!tar -xzf spark-2.2.0-bin-hadoop2.7.tgz
!mv spark-2.2.0-bin-hadoop2.7 apache-spark
!ls -al

In [ ]:
# Move it to the desired location. Make sure destination is writable.
!rm -rf /opt/apache-spark     # CHANGE DESTINATION PATH IF DESIRED.
!mv apache-spark /opt/        # CHANGE DESTINATION PATH IF DESIRED.

In [ ]:
# Now, in order to use the libraries, make sure the libs are included in $PATH and $PYTHONPATH.
# Add this to your .bashrc or .zshrc depending on your shell.
!export SPARK_HOME=/opt/apache-spark   # CHANGE DESTINATION PATH IF DESIRED.
!export PYTHONPATH="$SPARK_HOME/python/:$SPARK_HOME/python/lib/py4j-0.10.4-src.zip:$PYTHONPATH"
!export PATH="$SPARK_HOME/bin:$PATH"

## Apache Spark

Test your installation by importing the Python module. If you did everything correctly, this should not produce any errors.

In [1]:
import pyspark

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession

In the following sections we give a basic introduction to most of the concepts in Spark, and how (not) to use them. If you whish to implement something specific, you can always refer to the [documentation](https://spark.apache.org/docs/latest/quick-start.html).

As you saw in the lecture, Apache Spark provides two distinct concepts where computation resides. The first being the *driver*, which is basically the process which is responsible for issuing the tasks to the cluster, it is also used for *collecting* the results of the computation. The second, and final being the *executors*, which actually *execute* the tasks issues by the driver. Furthermore, an executor is also responsible for maintaining the state of the computation. For instance, if you issue a set of tasks from the driver process, and go for a coffee for instance, the (intermediate) result of the computation cannot be lost as long as the driver process is still alive.

Depending on how the cluster is configured (stand-alone, YARN, Mesos, ...), a different master-address has to be specified. The master will basically allocate the resources you requested and assign them to the requestor. In general, you have to specify 4 things before starting your Spark Application, i.e., the `application_name`, the `master_address` or master-mode (e.g., yarn-client, in this case you request yarn to schedule your resources), `num_executors` which obviously defines the number of executors you whish to use, and finally, `num_processes` which is the number of child-processes (you can view them as threads) which are run in a single executor. This option is advantegeous in the sense that the child processes within a single executor will share the same memory. As a result, these processes can share data really quickly (instead of reading data from a different node on the network).

In [2]:
# Define the name of your application.
application_name = "Large-Scale Distributed Systems: Spark Tutorial"

# Define the master address, for local mode this is local[*].
# If you don't want to use all cores on your machine please specify local[n].
master = "local[*]"
# Number of executors.
num_executors = 2
# Number of child processes per executors.
num_processes = 2

Now, using these options, we are able to construct a `SparkConf` object which tells the cluster how to allocate the resources, and specify the behaviour of the cluster. In the configuration specified below, you use the variables which are defines above, including the amount of memory you require on an executor level. In general, Spark thrives on the availability of RAM (more data can be kept in memory, so it can be processed faster).

A different option you can specify is a custom `Serializer`, the sole responsibility of the serializer is to serialize the memory to some kind of persistent medium whenever it doesn't fit in memory. Nevertheless, the way in-memory (heap) objects are serialized can differ from implementation. By default, Spark will use the Java serializer, which in some cases might not be sufficient because it fails to encode dependencies. That's why it is commonly a good-practice to use the Kryo Serializer. Furthermore, it is also faster than the Java serializer.

In [3]:
conf = SparkConf()
conf.set("spark.app.name", application_name)
conf.set("spark.master", master)
conf.set("spark.executor.cores", str(num_processes))
conf.set("spark.executor.instances", str(num_executors))
conf.set("spark.executor.memory", "2g")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

In general, the list of options you can specify are basically endless, and are application-dependent in order to obtain a good throughput of your processing application. If you would to look op a specific options, like e.g., increasing the *heartbeat* timeout, you could look it up in the [configuration documentation](https://spark.apache.org/docs/latest/configuration.html).

An other interesting concept in the Hadoop / Spark environment is the concept of *data locality*. Data locality, in principle, is a pretty powerfull concept in any distributed ecosystem which employs it, since it is an indirect side-effect of data replication on the distributed file system (in this case, HDFS), and of the processing which happens on top of that data. For instance, if you would schedule a certain set of tasks on the cluster, it would be rather inefficient to read a copy of the data on the disk of a remote machine, while a copy of the same data persists on a local disk. This concept can be extended even further on multiple levels, from the [Spark Tuning Guide](https://spark.apache.org/docs/latest/tuning.html#data-locality) (complete guide is recommended for optimizing your tasks):

- **```PROCESS_LOCAL```** data is in the same JVM as the running code. This is the best locality possible
- **```NODE_LOCAL```** data is on the same node. Examples might be in HDFS on the same node, or in another executor on the same node. This is a little slower than PROCESS_LOCAL because the data has to travel between processes
- **```NO_PREF```** data is accessed equally quickly from anywhere and has no locality preference
- **```RACK_LOCAL```** data is on the same rack of servers. Data is on a different server on the same rack so needs to be sent over the network, typically through a single switch
- **```ANY```** data is elsewhere on the network and not in the same rack

Of course, evaluating wheter data resides on the machine of the task, or on a machine on a different level (RACK_LOCAL or ANY) requires evaluating the current task graph, because maybe an executor is already processing this particular data and has it in memory, and if it does not reside there, it has to contact the HDFS journal to check where the copies of the data reside to actually pick to most efficient transfer. However, in some applications, (e.g., training neural networks concurrently), it is important that *mini-batches* arrive ASAP (as-fast-as-possible). As a result, checking for the most efficient data transfer might not be desirable. To solve this particular issue, Spark also provides an `spark.locality.wait` option where you can specify the amount of time you need to wait before Spark *launches a data-local task before giving up and launching it on a less-local node*.

In [4]:
# Since this notebook is (proably) run locally, locality 0 is desired.
conf.set("spark.locality.wait", "0")

The first thing a Spark program must do is to create a SparkSession object (which contains a SparkContext), which tells Spark how to access a cluster. To create a SparkSession we need to use the SparkConf object we previously created since it contains information about the application we want to run on the cluster.

In [5]:
# Allocate a Spark session based on the provided configuration and application name.
# Which may, or may not, already be created (getOrCreate()).
spark = SparkSession.builder.config(conf=conf).appName(application_name).getOrCreate()
# Create a shortcut for the SparkContext.
sc = spark.sparkContext

The main abstraction Spark provides is a resilient distributed dataset (RDD), which is a collection of elements partitioned across the nodes of the cluster that can be operated on in parallel. RDDs are created by starting with a file in the Hadoop file system (or any other Hadoop-supported file system), or an existing list in the driver program, and transforming it. Users may also ask Spark to persist an RDD in memory, allowing it to be reused efficiently across parallel operations. Finally, RDDs automatically recover from node failures.

## BetterPiSpark

Traditionally, starting users run the `PiSpark` example which basically  estimates Pi using the sampling method. However, the way the example is implemented is actually a non-efficient method since it relies on a list of indexes which scales lineary with the input. Meaning, for every item in the list the method will compute one sample. This is not a very efficient way to do this, you can imagine if you are processing images that you will not be able to load TBs of image data in the RAM of your driver program, and then parallelize to the cluster. As a result, we will construct a `BetterPiSpark` example, which I will guide you through.

Given our resource allocation defined in the `SparkConf` object, we know the total number of concurrent processes or tasks that will be running whenever we submit some jobs in the cluster. This is basically `num_executors` $*$ `num_processes`.

In [6]:
num_parallel_tasks = num_executors * num_processes

A way to generate the samples more efficiently, would be to tell every concurrent task to generate a specific number number of random samples. To do this, we first need to construct an RDD which holds the number of samples that every task has to generate.

In [7]:
# Define the total number of samples that has to be generated.
total_samples = 100000000
# Define the number of samples per process.
samples_per_process = int(total_samples / num_parallel_tasks)
samples = [samples_per_process] * num_parallel_tasks

Now we have a list in the driver program which specifies the number of samples per process, we have to parallelize that list over the cluster.

In [8]:
rdd = sc.parallelize(samples)

In order to ensure that the RDD really holds 4 items, we can call a simple `count` operation. However, note that compared to `len(samples)` this operation is quite slow (for this amount of data) because the tasks have to be scheduled among all executors and processes, and collected afterwards.

**Interesting observation:** Running this cell twice will (probably) be faster than the initial try, why?

In [10]:
rdd.count() == len(samples)

True

## Stopping the SparkSession

In [ ]:
# Finally, stop the Spark session to free up the allocated resources.
spark.stop()